In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
## aligining discrimination repports with next day survey and sensor data
# NOTE this is temporary; the reimplementation of the R analysis code should have this
#data_file = '/Users/yasaman/UWEXP/analysis-scripts/combined/results/bigtable_UWonly.csv'
data_file = '/Users/yasaman/UWEXP/analysis-scripts/combined/results/bigtable_UWonly+step.csv'
#data_file = '/Users/yasaman/UWEXP/analysis-scripts/combined/results/aggregated-all-cmu.csv'
#data_file = '/Users/yasaman/UWEXP/analysis-scripts/surveys/results/emaaggregation/aggregated-horizontal-numVal-internalID.csv'
PIDs2remove = [18, 26, 27, 41, 53, 56, 69, 71, 81, 83, 85, 89, 100, 101, 107, 112, 114, 119, 121, 129, 131, 133, 135, 
               139, 141, 147, 152, 164, 182, 192, 197, 200, 208]
PIDs2remove_cmu = [524]
offset = '2017-12-31'
PIDs2remove_sensor = [18, 29, 46, 48, 68, 71, 74, 81, 83, 95, 103, 113, 133, 151, 152, 159, 173, 178, 182, 184, 196, 
                      197, 199, 200, 203, 204]
day_sensor_start = 29 # Jan 29
day_sensor_end = 156 # Jun 05

In [ ]:
data = pd.read_csv(data_file, na_values = [' '])

In [ ]:
discrimination_categories = { 
  1: 'national', 
  2: 'gender', 
  3: 'orientation', 
  4: 'intelligence', 
  5: 'major', 
  6: 'learning', 
  7: 'income', 
  8: 'age', 
  9: 'religion', 
 10: 'disability', 
 11: 'height', 
 12: 'weight', 
 13: 'appearance'
}

In [ ]:
cmu_discrimination_breakdown = data[data['unfair_type'] != 14].groupby(by=['unfair_type']).size()
cmu_discrimination_breakdown = cmu_discrimination_breakdown.rename(discrimination_categories)
cmu_discrimination_breakdown = cmu_discrimination_breakdown.sort_values(ascending=False)

In [ ]:
cmu_discrimination_breakdown = data[(data['unfair_type'] != 14) & (data['Gender_MFO_B1'] == 2)].groupby(by=['unfair_type']).size()
cmu_discrimination_breakdown = cmu_discrimination_breakdown.rename(discrimination_categories)
cmu_discrimination_breakdown = cmu_discrimination_breakdown.sort_values(ascending=False)

In [ ]:
fig, ax = plt.subplots(1, 1,  figsize=(8, 6))
cmu_discrimination_breakdown.plot(kind='bar', ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
plt.savefig('/Users/yasaman/Downloads/cmu-discrimination-breakdown-women.png', format = 'png', bbox_inches = 'tight')

In [ ]:
data['day'] = (pd.to_datetime(data['date']) - pd.to_datetime(offset)).dt.days
columns = list(data.columns)
columns.remove('day')
columns.insert(columns.index('date')+1, 'day')
data = data[columns]

In [ ]:
data.sort_values(by=['PID', 'date'], ascending=[True, True], inplace=True)
after_days = 7
for i in range(1, after_days):
    data['discriminated_shifted_{}'.format(i)] = data['discriminated'].shift(periods=i)
columns = list(data.columns)
for i in range(1, after_days):
    columns.remove('discriminated_shifted_{}'.format(i))
ind = columns.index('discriminated')
for i in range(1, after_days):
    columns.insert(ind+i, 'discriminated_shifted_{}'.format(i))
data = data[columns]

In [ ]:
data['same_day'] = data['discriminated'].map({'YES': 1, 'NO': 0}, na_action='ignore')
for i in range(1, after_days):
    if i == 1:
        data['next_day'] = data['discriminated_shifted_{}'.format(i)].map({'YES': 1, 'NO': 0}, na_action='ignore')
    else:
        data['next_{}days'.format(i)] = data['discriminated_shifted_{}'.format(i)].map({'YES': 1, 'NO': 0}, na_action='ignore')
columns = list(data.columns)
columns.remove('same_day')
for i in range(1, after_days):
    if i == 1:
        columns.remove('next_day')
    else:
        columns.remove('next_{}days'.format(i))
columns.insert(columns.index('discriminated')+1, 'same_day')
for i in range(1, after_days):
    if i == 1:
        columns.insert(columns.index('discriminated_shifted_1')+1, 'next_day')
    else:
        columns.insert(columns.index('discriminated_shifted_{}'.format(i))+1, 'next_{}days'.format(i))
data = data[columns]

In [ ]:
data.sort_values(by=['PID', 'date'], ascending=[True, True], inplace=True)
before_days = 7
for i in range(1, before_days):
    data['discriminated_shifted_-{}'.format(i)] = data['discriminated'].shift(periods=-i)
columns = list(data.columns)
for i in range(1, before_days):
    columns.remove('discriminated_shifted_-{}'.format(i))
ind = columns.index('discriminated')
for i in range(1, before_days):
    columns.insert(ind, 'discriminated_shifted_-{}'.format(i))
data = data[columns]

In [ ]:
for i in range(1, before_days):
    if i == 1:
        data['day_before'] = data['discriminated_shifted_-{}'.format(i)].map({'YES': 1, 'NO': 0}, na_action='ignore')
    else:
        data['{}days_before'.format(i)] = data['discriminated_shifted_-{}'.format(i)].map({'YES': 1, 'NO': 0}, na_action='ignore')
columns = list(data.columns)
for i in range(1, before_days):
    if i == 1:
        columns.remove('day_before')
    else:
        columns.remove('{}days_before'.format(i))
for i in range(1, before_days):
    if i == 1:
        columns.insert(columns.index('discriminated_shifted_-1')+1, 'day_before')
    else:
        columns.insert(columns.index('discriminated_shifted_-{}'.format(i))+1, '{}days_before'.format(i))
data = data[columns]

In [ ]:
# PID524 does not have any evening or weekly EMA containing the discrimiantion question. It is thus removed
data = data[~data['PID'].isin(PIDs2remove_cmu)]

In [ ]:
data['discriminated_num'] = data.groupby(by=['PID'])['same_day'].transform(sum)
data['discriminated_yn'] = (data['discriminated_num'] > 0).map({True: 1, False: 0}, na_action='ignore')
data['discriminated_count'] = data.groupby(by=['PID'])['same_day'].transform('count')
data['discriminated_rate'] = data.apply(lambda x: x['discriminated_num']/x['discriminated_count'], axis = 1)
columns = list(data.columns)
columns.remove('discriminated_num')
columns.remove('discriminated_yn')
columns.remove('discriminated_count')
columns.remove('discriminated_rate')
ind = columns.index('discriminated')
columns.insert(ind, 'discriminated_rate')
columns.insert(ind, 'discriminated_count')
columns.insert(ind, 'discriminated_num')
columns.insert(ind, 'discriminated_yn')
data = data[columns]

In [ ]:
def wrt_discrimination(row, after_days, before_days):
    if row['same_day'] == 1:
        return 0
    for i in range(1, after_days):
        if i == 1:
            col = 'next_day'
        else:
            col = 'next_{}days'.format(i)   
        if row[col] == 1:
            return i
    for i in range(1, before_days):
        if i == 1:
            col = 'day_before'
        else:
            col = '{}days_before'.format(i)
        if row[col] == 1:
            return -i
    return None
#data = pd.DataFrame({'day': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
#           'discriminated': [None, 'NO', None, None, None, 'YES', None, None, 'YES', None, None, None, 'NO', 'YES', 'YES', 'NO', 'NO', 'NO', 'NO', None, None, 'NO']})

In [ ]:
data['day_of_discrimination'] = data.apply(lambda x: wrt_discrimination(x, after_days, before_days), axis=1)
columns = list(data.columns)
columns.remove('day_of_discrimination')
ind = columns.index('discriminated_yn')
columns.insert(ind, 'day_of_discrimination')
data = data[columns]

In [ ]:
data['CES_D_delta'] = data['CES_D_POST'] - data['CES_D_B2']
#data['STAI_delta'] = data['STAI_POST'] - data['STAI_B2'] # comment out for CMU
data['PSS_delta'] = data['PSS_POST'] - data['PSS_B2']
data['UCLA_Loneliness_delta'] = data['UCLA_Loneliness_POST'] - data['UCLA_Loneliness_B2']
columns = list(data.columns)
columns.remove('CES_D_delta')
#columns.remove('STAI_delta') # comment out for CMU
columns.remove('PSS_delta')
columns.remove('UCLA_Loneliness_delta')
columns.insert(columns.index('CES_D_POST')+1, 'CES_D_delta')
#columns.insert(columns.index('STAI_POST')+1, 'STAI_delta') # comment out for CMU
columns.insert(columns.index('PSS_POST')+1, 'PSS_delta')
columns.insert(columns.index('UCLA_Loneliness_POST')+1, 'UCLA_Loneliness_delta')
data = data[columns]

In [ ]:
#affect_items = ['anxious', 'depressed', 'frustrated', 'overwhelmed', 'lonely', 'happy', 'connected'] # comment out for CMU
affect_items = ['anxious', 'depressed', 'overwhelmed', 'lonely', 'happy', 'connected'] # uncomment for CMU
for affect in affect_items:
    # data[affect] = data['feel_'+affect+'_weekly'] + data['feel_'+affect+'_morning']
    # NOTE this generates a blank column as the result would be NaN if either columns is NaN which is always the case
    # data[affect] = data[['feel_'+affect+'_weekly','feel_'+affect+'_morning']].sum(axis=1)
    # NOTE this will generate columns with a lot of zeros when both weekly and morning columns are NaN. We don't want this
    data[affect] = data['feel_'+affect+'_weekly'].combine_first(data['feel_'+affect+'_morning'])
# for next day analysis this means we look at either weekly or morning ratings

In [ ]:
columns = list(data.columns)
for affect in affect_items:
    columns.remove(affect)
    columns.insert(columns.index('feel_anxious_weekly'), affect)
data = data[columns]

In [ ]:
within_cols = affect_items + ['feel_'+affect+'_evening' for affect in affect_items] + sensor_metrics
within = data.groupby(by='PID')[within_cols].transform(lambda x: x - x.mean()+1)
within.rename({col: col+'_within' for col in within_cols}, axis='columns', inplace=True)

In [ ]:
data = pd.concat([data, within], axis=1)

In [ ]:
data.to_csv('/Users/yasaman/UWEXP/analysis-scripts/combined/results/bigtable_UWonly+step_discriminated_7days_aligned+numeric+stats+delta+affectAgg+within.csv', index=False)
#data.to_csv('/Users/yasaman/UWEXP/analysis-scripts/combined/results/cmu_discriminated_7days_aligned+numeric+stats+delta+affectAgg.csv', index=False)

In [ ]:
x2data = data.groupby(by=['PID']).first()[['Gender_MFO_B1', 'Engineer_B1', 'discriminated_yn']]

In [ ]:
print('sample breakdown by gender', x2data.groupby(by='Gender_MFO_B1').size())
#print(x2data.groupby(by='Gender_MFO_B1')['discriminated_yn'].sum())
print('breakdown of reports of discrimination by gender', 
        x2data[x2data['discriminated_yn'] == 1].groupby(by='Gender_MFO_B1').size())
print('breakdown of no reports of discrimination by gender', 
        x2data[x2data['discriminated_yn'] == 0].groupby(by='Gender_MFO_B1').size())

In [ ]:
print('sample breakdown by engineering status', x2data.groupby(by='Engineer_B1').size())
#print('breakdown for by engineering status', x2data.groupby(by='Engineer_B1')['discriminated_yn'].sum())
print('breakdown of reports of discrimination by engineering status', 
        x2data[x2data['discriminated_yn'] == 1].groupby(by='Engineer_B1').size())
print('breakdown of no reports of discrimination by engineering status',
        x2data[x2data['discriminated_yn'] == 0].groupby(by='Engineer_B1').size())

In [ ]:
print('total reports of discrimination', data[(data['discriminated'] == 'YES')].shape[0])
print('total reports of discrimination by women', data[(data['discriminated'] == 'YES') & (data['Gender_MFO_B1'] == 2)].shape[0])
print('total reports of discrimination by men', data[(data['discriminated'] == 'YES') & (data['Gender_MFO_B1'] == 1)].shape[0])

In [ ]:
print('total reports of discrimination', data[(data['discriminated'] == 'YES')].shape[0])
print('total reports of discrimination by engineers', data[(data['discriminated'] == 'YES') & (data['Engineer_B1'] == 1)].shape[0])
print('total reports of discrimination by non-engineers', data[(data['discriminated'] == 'YES') & (data['Engineer_B1'] == 0)].shape[0])

In [ ]:
data_clean = data[~data['PID'].isin(PIDs2remove)]

In [ ]:
pss_means = data_clean[['PSS_B2', 'PSS_MID', 'PSS_POST']].mean()
uls_means = data_clean[['UCLA_Loneliness_B2', 'UCLA_Loneliness_MID', 'UCLA_Loneliness_POST']].mean()

In [ ]:
['{:.2f}'.format(item) for item in list(pss_means)]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
pss_means.plot(kind='bar', ax=ax)
ax.set_xticklabels(['Baseline', 'Between Quarters', 'Post'], rotation=45, ha='right')
ax.set_ylabel('Perceived Stress Scores (PSS)')
bars = ax.patches
labels = ['{:.2f}'.format(item) for item in list(pss_means)]
for bar, label in zip(bars, labels):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, height - 1.5, label, ha='center', va='bottom')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
uls_means.plot(kind='bar', ax=ax)
ax.set_xticklabels(['Baseline', 'Between Quarters', 'Post'], rotation=45, ha='right')
ax.set_ylabel('UCLA Loneliness Scores (ULS)')
bars = ax.patches
labels = ['{:.2f}'.format(item) for item in list(uls_means)]
for bar, label in zip(bars, labels):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, height - 3.5, label, ha='center', va='bottom')

In [ ]:
data_clean.to_csv('/Users/yasaman/UWEXP/analysis-scripts/combined/results/bigtable_UWonly_176+step_discriminated_7days_aligned+numeric+stats+delta+affectAgg.csv', index=False)

In [ ]:
data_sensor = data[~data['PID'].isin(PIDs2remove_sensor)]
data_sensor = data_sensor[(data_sensor['day'] >= day_sensor_start) & (data_sensor['day'] <= day_sensor_end)] 
# NOTE: we only look at sensor data betwee Jan 29 (day 29) and Jun 05 (day 156) 

In [ ]:
data_sensor.to_csv('/Users/yasaman/UWEXP/analysis-scripts/combined/results/bigtable_UWonly+step_176+sensor_discriminated_nexday_aligned+numeric+stats+delta+affectAgg.csv', index=False)

In [ ]:
postoutcomes = ['CES_D_POST', 'STAI_POST', 'PSS_POST', 'UCLA_Loneliness_POST']
postoutcomes_PIDs2remove = data_PIDs2remove.groupby(by=['PID']).first()[postoutcomes]
postoutcomes_PIDs2remove

In [ ]:
#data['Gender_MFO_POST'].unique() # returns array([ 1.,  2., nan,  0.])
#data.groupby(by=['Gender_MFO_POST'])['Gender_MFO_POST'].size() # returns 0 --> 166, 1 --> 10126, 2 --> 18592
#data[data['Gender_MFO_B1'] != data['Gender_MFO_POST']][['PID', 'Gender_MFO_B1', 'Gender_MFO_POST']] # three people how have B1 have NaN in post
data[data['Gender_MFO_B1'] != data['Gender_MFO_POST']]['PID'].unique() # returns array([103, 125, 190])

In [ ]:
outcomes = ['BDI_II_POST', 
            'CES_D_B2', 'CES_D_POST', 
            'STAI_B2', 'STAI_POST', # comment out for CMU
            'PSS_B2', 'PSS_POST', 
            'UCLA_Loneliness_B2', 'UCLA_Loneliness_POST']
outcomes = data.groupby(by=['PID']).first()[outcomes]

In [ ]:
outcomes[outcomes.isnull().any(axis=1)] # TO-DO why don't we have POST values for these people?

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
sns.distplot(outcomes[outcomes['BDI_II_POST'].notnull()]['BDI_II_POST'], bins=100, color='b', ax=ax)
ax.set_xlabel('BDI_II depression scores')

In [ ]:
sns.distplot(outcomes['CES_D_B2'], bins=100, color='k')
sns.distplot(outcomes[outcomes['CES_D_POST'].notnull()]['CES_D_POST'], bins=100, color='b')

In [ ]:
sns.distplot(outcomes['STAI_B2'], bins=100, color='k')
sns.distplot(outcomes[outcomes['STAI_POST'].notnull()]['STAI_POST'], bins=100, color='b')

In [ ]:
sns.distplot(outcomes['PSS_B2'], bins=100, color='k')
sns.distplot(outcomes[outcomes['PSS_POST'].notnull()]['PSS_POST'], bins=100, color='b')

In [ ]:
sns.distplot(outcomes['UCLA_Loneliness_B2'], bins=100, color='k')
sns.distplot(outcomes[outcomes['UCLA_Loneliness_POST'].notnull()]['UCLA_Loneliness_POST'], bins=100, color='b')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 4))
outcomes.boxplot(column=['CES_D_B2', 'CES_D_POST'], ax=ax, widths=0.5)
#ax.set_aspect(0.05)
ax.set_xticklabels(['Pre', 'Post'])
ax.set_ylabel('CES-D Depression Score')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 4))
outcomes.boxplot(column=['STAI_B2', 'STAI_POST'], ax=ax, widths=0.5)
#ax.set_aspect(0.05)
ax.set_xticklabels(['Pre', 'Post'])
ax.set_ylabel('STAI Anxiety Score')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 4))
outcomes.boxplot(column=['PSS_B2', 'PSS_POST'], ax=ax, widths=0.5)
#ax.set_aspect(0.05)
ax.set_xticklabels(['Pre', 'Post'])
ax.set_ylabel('PSS Stress Score')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 4))
outcomes.boxplot(column=['UCLA_Loneliness_B2', 'UCLA_Loneliness_POST'], ax=ax, widths=0.5)
#ax.set_aspect(0.05)
ax.set_xticklabels(['Pre', 'Post'])
ax.set_ylabel('UCLA Loneliness Score')

In [ ]:
outcomes_delta = ['CES_D_delta', 'STAI_delta', 'PSS_delta', 'UCLA_Loneliness_delta']
outcomes_delta = data.groupby(by=['PID']).first()[outcomes_delta]

In [ ]:
sns.distplot(outcomes_delta[outcomes_delta['CES_D_delta'].notnull()]['CES_D_delta'], bins=100, color='k')

In [ ]:
sns.distplot(outcomes_delta[outcomes_delta['STAI_delta'].notnull()]['STAI_delta'], bins=100, color='k')

In [ ]:
sns.distplot(outcomes_delta[outcomes_delta['PSS_delta'].notnull()]['PSS_delta'], bins=100, color='k')

In [ ]:
sns.distplot(outcomes_delta[outcomes_delta['UCLA_Loneliness_delta'].notnull()]['UCLA_Loneliness_delta'], bins=100, color='k')

In [ ]:
moderators = ['ISEL_APPRAISEL_POST', 'ISEL_BELONGING_POST', 'ISEL_TANGIBLE_POST', 
              'K2way_SSS_POST', 
              'K2way_SSS_REmotional_POST','K2way_SSS_GEmotional_POST',
              'K2way_SSS_RInstrumental_POST','K2way_SSS_GInstrumental_POST',
              'K2way_SSS_Receive_POST','K2way_SSS_Give_POST',
              'MAAS_POST', 
              'ERQ_POST', 
              'ERQ_Reappraisal_POST', 'ERQ_Suppression_POST',
              'BRS_POST',
              'QualtricsSF12Mental_POST', 'QualtricsSF12Physical_POST', # NOTE: these should be replaced with SF12Mental_POST and SF12Physical_POST
              'CHIPS_POST']
moderators = data.groupby(by=['PID']).first()[moderators]

In [ ]:
sns.pairplot(moderators[moderators.notnull().all(axis=1)], diag_kind='kde', plot_kws={'alpha': 0.2})

In [ ]:
outmod = pd.merge(outcomes, moderators, left_index= True, right_index=True, how='inner')
outmod = outmod[outmod.notnull().all(axis=1)]

In [ ]:
sns.pairplot(outmod[['CES_D_POST', 'CHIPS_POST', 'MAAS_POST', 'ERQ_POST']], diag_kind='kde', plot_kws={'alpha': 0.2})

In [ ]:
long_term = list(outcomes.columns) + list(moderators.columns) + ['discriminated_yn', 'discriminated_rate']
long_term = data.groupby(by=['PID']).first()[long_term]

In [ ]:
long_term = long_term[~long_term.isnull().any(axis=1)]

In [ ]:
M = long_term[long_term['discriminated_rate'] > 0]['discriminated_rate'].mean()
std = long_term[long_term['discriminated_rate'] > 0]['discriminated_rate'].std()

In [ ]:
print(long_term[long_term['discriminated_rate'] > (M+5*std)]['discriminated_yn'].sum())
print(long_term[long_term['discriminated_rate'] > 0.6]['discriminated_yn'].sum())

In [ ]:
print(M+2*std)

In [ ]:
sns.regplot('discriminated_rate', 'CES_D_POST', data=long_term)

In [ ]:
sns.regplot('discriminated_rate', 'CES_D_POST', data=long_term[long_term['discriminated_rate'] < 0.5])

In [ ]:
sns.regplot('discriminated_rate', 'CES_D_POST', data=long_term[(long_term['discriminated_rate'] < 0.5) & (long_term['discriminated_rate'] > 0)])

In [ ]:
sns.lmplot(x="CES_D_POST", y="BRS_POST", hue="discriminated_yn", data=long_term[(long_term['discriminated_rate'] < 0.5) & (long_term['discriminated_rate'] > 0)])

In [ ]:
sns.lmplot(x="CES_D_POST", y="K2way_SSS_POST", hue="discriminated_yn", data=long_term[(long_term['discriminated_rate'] < 0.5) & (long_term['discriminated_rate'] > 0)])

In [ ]:
sns.lmplot(x="CES_D_POST", y="BRS_POST", hue="discriminated_yn", data=long_term[(long_term['discriminated_rate'] < 0.5)])

In [ ]:
sns.lmplot(x="CES_D_POST", y="K2way_SSS_POST", hue="discriminated_yn", data=long_term[(long_term['discriminated_rate']<0.15)])

In [ ]:
sns.regplot('discriminated_rate', 'STAI_POST', data=long_term)

In [ ]:
sns.regplot('discriminated_rate', 'STAI_POST', data=long_term[long_term['discriminated_rate'] < 0.5])

In [ ]:
sns.regplot('discriminated_rate', 'STAI_POST', data=long_term[(long_term['discriminated_rate'] < 0.5) & (long_term['discriminated_rate'] > 0.05)])

In [ ]:
sns.regplot('discriminated_rate', 'PSS_POST', data=long_term)

In [ ]:
sns.regplot('discriminated_rate', 'PSS_POST', data=long_term[long_term['discriminated_rate'] < 0.5])

In [ ]:
sns.regplot('discriminated_rate', 'PSS_POST', data=long_term[(long_term['discriminated_rate'] < 0.5) & (long_term['discriminated_rate'] > 0.05)])

In [ ]:
sns.regplot('discriminated_rate', 'UCLA_Loneliness_POST', data=long_term)

In [ ]:
sns.regplot('discriminated_rate', 'UCLA_Loneliness_POST', data=long_term[long_term['discriminated_rate'] < 0.5])

In [ ]:
sns.regplot('discriminated_rate', 'UCLA_Loneliness_POST', data=long_term[(long_term['discriminated_rate'] < 0.5) & (long_term['discriminated_rate'] > 0.05)])

In [ ]:
sns.lmplot(x="CES_D_POST", y="K2way_SSS_POST", hue="discriminated_yn", data=long_term)

In [ ]:
ax = sns.lmplot(x="CES_D_POST", y="K2way_SSS_POST", hue="discriminated_yn", data=long_term[long_term['discriminated_rate'] < 0.5])
ax.set(xlabel='CES-D Depression Score', ylabel='2way SSS Social Support')
ax._legend.set_title('discriminated?')
new_labels = ['NO', 'YES']
for t, l in zip(ax._legend.texts, new_labels): t.set_text(l)

In [ ]:
affect_cols = affect_items + ['feel_'+affect+'_evening' for affect in affect_items]
summary_affect = data.groupby(by=['day_of_discrimination'])[affect_cols].mean()
for affect in affect_items:
    summary_affect.loc[0, affect] = summary_affect.loc[0, 'feel_'+affect+'_evening']
summary_affect = summary_affect[affect_items]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
summary_affect[['depressed', 'frustrated']].plot(ax=ax)
ax.set_xlabel('Days around Discrimination')
ax.set_ylabel('Strength of Feelings')
ax.legend(loc='upper right')

In [ ]:
affect_cols = [affect+'_within' for affect in affect_items] + ['feel_'+affect+'_evening_within' for affect in affect_items]
summary_affect = data.groupby(by=['day_of_discrimination'])[affect_cols].mean()
for affect in affect_items:
    summary_affect.loc[0, affect+'_within'] = summary_affect.loc[0, 'feel_'+affect+'_evening_within']
summary_affect = summary_affect[[affect+'_within' for affect in affect_items]]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
summary_affect[['depressed_within', 'frustrated_within']].plot(ax=ax)
ax.set_xlabel('Days around Discrimination')
ax.set_ylabel('Strength of Feelings')
ax.legend(loc='upper right')

In [ ]:
sensor_metrics = ['activity_count_changes_allday', 
                 'activity_count_changes_morning',
                 'activity_count_changes_afternoon',
                 'activity_count_changes_evening',
                 'activity_count_changes_night',
                 'activity_number_of_activities_allday',
                 'activity_number_of_activities_morning',
                 'activity_number_of_activities_afternoon',
                 'activity_number_of_activities_evening',
                 'activity_number_of_activities_night',
                 'battery_num_rows_battery_allday',
                 'battery_num_rows_battery_morning',
                 'battery_num_rows_battery_afternoon',
                 'battery_num_rows_battery_evening',
                 'battery_num_rows_battery_night',
                 'battery_length_of_charge_minutes_allday',
                 'battery_length_of_charge_minutes_morning',
                 'battery_length_of_charge_minutes_afternoon',
                 'battery_length_of_charge_minutes_evening',
                 'battery_length_of_charge_minutes_night',
                 'bluetooth_num_scans_of_most_frequent_device_allday',
                 'bluetooth_num_scans_of_most_frequent_device_morning',
                 'bluetooth_num_scans_of_most_frequent_device_afternoon',
                 'bluetooth_num_scans_of_most_frequent_device_evening',
                 'bluetooth_num_scans_of_most_frequent_device_night',
                 'bluetooth_number_samples_bluetooth_allday',
                 'bluetooth_number_samples_bluetooth_morning',
                 'bluetooth_number_samples_bluetooth_afternoon',
                 'bluetooth_number_samples_bluetooth_evening',
                 'bluetooth_number_samples_bluetooth_night',
                 'bluetooth_sum_num_scans_of_all_devices_of_self_allday',
                 'bluetooth_sum_num_scans_of_all_devices_of_self_morning',
                 'bluetooth_sum_num_scans_of_all_devices_of_self_afternoon',
                 'bluetooth_sum_num_scans_of_all_devices_of_self_evening',
                 'bluetooth_sum_num_scans_of_all_devices_of_self_night',
                 'bluetooth_avg_num_scans_of_all_devices_of_others_allday',
                 'bluetooth_avg_num_scans_of_all_devices_of_others_morning',
                 'bluetooth_avg_num_scans_of_all_devices_of_others_afternoon',
                 'bluetooth_avg_num_scans_of_all_devices_of_others_evening',
                 'bluetooth_avg_num_scans_of_all_devices_of_others_night',
                 'bluetooth_std_num_scans_of_all_devices_of_others_allday',
                 'bluetooth_std_num_scans_of_all_devices_of_others_morning',
                 'bluetooth_std_num_scans_of_all_devices_of_others_afternoon',
                 'bluetooth_std_num_scans_of_all_devices_of_others_evening',
                 'bluetooth_std_num_scans_of_all_devices_of_others_night',
                 'bluetooth_num_scans_of_least_frequent_device_of_self_allday',
                 'bluetooth_num_scans_of_least_frequent_device_of_self_morning',
                 'bluetooth_num_scans_of_least_frequent_device_of_self_afternoon',
                 'bluetooth_num_scans_of_least_frequent_device_of_self_evening',
                 'bluetooth_num_scans_of_least_frequent_device_of_self_night',
                 'bluetooth_number_unique_devices_of_others_allday',
                 'bluetooth_number_unique_devices_of_others_morning',
                 'bluetooth_number_unique_devices_of_others_afternoon',
                 'bluetooth_number_unique_devices_of_others_evening',
                 'bluetooth_number_unique_devices_of_others_night',
                 'bluetooth_num_scans_of_most_frequent_device_of_self_allday',
                 'bluetooth_num_scans_of_most_frequent_device_of_self_morning',
                 'bluetooth_num_scans_of_most_frequent_device_of_self_afternoon',
                 'bluetooth_num_scans_of_most_frequent_device_of_self_evening',
                 'bluetooth_num_scans_of_most_frequent_device_of_self_night',
                 'bluetooth_std_num_scans_of_all_devices_of_self_allday',
                 'bluetooth_std_num_scans_of_all_devices_of_self_morning',
                 'bluetooth_std_num_scans_of_all_devices_of_self_afternoon',
                 'bluetooth_std_num_scans_of_all_devices_of_self_evening',
                 'bluetooth_std_num_scans_of_all_devices_of_self_night',
                 'bluetooth_num_scans_of_least_frequent_device_allday',
                 'bluetooth_num_scans_of_least_frequent_device_morning',
                 'bluetooth_num_scans_of_least_frequent_device_afternoon',
                 'bluetooth_num_scans_of_least_frequent_device_evening',
                 'bluetooth_num_scans_of_least_frequent_device_night',
                 'bluetooth_sum_num_scans_of_all_devices_of_others_allday',
                 'bluetooth_sum_num_scans_of_all_devices_of_others_morning',
                 'bluetooth_sum_num_scans_of_all_devices_of_others_afternoon',
                 'bluetooth_sum_num_scans_of_all_devices_of_others_evening',
                 'bluetooth_sum_num_scans_of_all_devices_of_others_night',
                 'bluetooth_num_scans_of_least_frequent_device_of_others_allday',
                 'bluetooth_num_scans_of_least_frequent_device_of_others_morning',
                 'bluetooth_num_scans_of_least_frequent_device_of_others_afternoon',
                 'bluetooth_num_scans_of_least_frequent_device_of_others_evening',
                 'bluetooth_num_scans_of_least_frequent_device_of_others_night',
                 'bluetooth_number_unique_devices_of_self_allday',
                 'bluetooth_number_unique_devices_of_self_morning',
                 'bluetooth_number_unique_devices_of_self_afternoon',
                 'bluetooth_number_unique_devices_of_self_evening',
                 'bluetooth_number_unique_devices_of_self_night',
                 'bluetooth_number_unique_devices_allday',
                 'bluetooth_number_unique_devices_morning',
                 'bluetooth_number_unique_devices_afternoon',
                 'bluetooth_number_unique_devices_evening',
                 'bluetooth_number_unique_devices_night',
                 'bluetooth_avg_num_scans_of_all_devices_of_self_allday',
                 'bluetooth_avg_num_scans_of_all_devices_of_self_morning',
                 'bluetooth_avg_num_scans_of_all_devices_of_self_afternoon',
                 'bluetooth_avg_num_scans_of_all_devices_of_self_evening',
                 'bluetooth_avg_num_scans_of_all_devices_of_self_night',
                 'bluetooth_num_scans_of_most_frequent_device_of_others_allday',
                 'bluetooth_num_scans_of_most_frequent_device_of_others_morning',
                 'bluetooth_num_scans_of_most_frequent_device_of_others_afternoon',
                 'bluetooth_num_scans_of_most_frequent_device_of_others_evening',
                 'bluetooth_num_scans_of_most_frequent_device_of_others_night',
                 'calls_number_missed_calls_allday',
                 'calls_number_missed_calls_morning',
                 'calls_number_missed_calls_afternoon',
                 'calls_number_missed_calls_evening',
                 'calls_number_missed_calls_night',
                 'calls_duration_outgoing_calls_seconds_allday',
                 'calls_duration_outgoing_calls_seconds_morning',
                 'calls_duration_outgoing_calls_seconds_afternoon',
                 'calls_duration_outgoing_calls_seconds_evening',
                 'calls_duration_outgoing_calls_seconds_night',
                 'calls_number_rows_calls_allday',
                 'calls_number_rows_calls_morning',
                 'calls_number_rows_calls_afternoon',
                 'calls_number_rows_calls_evening',
                 'calls_number_rows_calls_night',
                 'calls_number_incoming_calls_allday',
                 'calls_number_incoming_calls_morning',
                 'calls_number_incoming_calls_afternoon',
                 'calls_number_incoming_calls_evening',
                 'calls_number_incoming_calls_night',
                 'calls_duration_incoming_calls_seconds_allday',
                 'calls_duration_incoming_calls_seconds_morning',
                 'calls_duration_incoming_calls_seconds_afternoon',
                 'calls_duration_incoming_calls_seconds_evening',
                 'calls_duration_incoming_calls_seconds_night',
                 'calls_number_outgoing_calls_allday',
                 'calls_number_outgoing_calls_morning',
                 'calls_number_outgoing_calls_afternoon',
                 'calls_number_outgoing_calls_evening',
                 'calls_number_outgoing_calls_night',
                 'locations_location_variance_allday',
                 'locations_location_variance_morning',
                 'locations_location_variance_afternoon',
                 'locations_location_variance_evening',
                 'locations_location_variance_night',
                 'locations_location_variance_log_allday',
                 'locations_location_variance_log_morning',
                 'locations_location_variance_log_afternoon',
                 'locations_location_variance_log_evening',
                 'locations_location_variance_log_night',
                 'locations_location_entropy_local_allday',
                 'locations_location_entropy_local_morning',
                 'locations_location_entropy_local_afternoon',
                 'locations_location_entropy_local_evening',
                 'locations_location_entropy_local_night',
                 'locations_location_entropy_normalized_local_allday',
                 'locations_location_entropy_normalized_local_morning',
                 'locations_location_entropy_normalized_local_afternoon',
                 'locations_location_entropy_normalized_local_evening',
                 'locations_location_entropy_normalized_local_night',
                 'locations_location_entropy_normalized_allday',
                 'locations_location_entropy_normalized_morning',
                 'locations_location_entropy_normalized_afternoon',
                 'locations_location_entropy_normalized_evening',
                 'locations_location_entropy_normalized_night',
                 'locations_location_entropy_allday',
                 'locations_location_entropy_morning',
                 'locations_location_entropy_afternoon',
                 'locations_location_entropy_evening',
                 'locations_location_entropy_night',
                 'locations_radius_of_gyration_allday',
                 'locations_radius_of_gyration_morning',
                 'locations_radius_of_gyration_afternoon',
                 'locations_radius_of_gyration_evening',
                 'locations_radius_of_gyration_night',
                 'locations_circadian_movement_allday',
                 'locations_circadian_movement_morning',
                 'locations_circadian_movement_afternoon',
                 'locations_circadian_movement_evening',
                 'locations_circadian_movement_night',
                 'locations_number_location_transitions_allday',
                 'locations_number_location_transitions_morning',
                 'locations_number_location_transitions_afternoon',
                 'locations_number_location_transitions_evening',
                 'locations_number_location_transitions_night',
                 'locations_number_of_clusters_local_allday',
                 'locations_number_of_clusters_local_morning',
                 'locations_number_of_clusters_local_afternoon',
                 'locations_number_of_clusters_local_evening',
                 'locations_number_of_clusters_local_night',
                 'locations_number_of_clusters_allday',
                 'locations_number_of_clusters_morning',
                 'locations_number_of_clusters_afternoon',
                 'locations_number_of_clusters_evening',
                 'locations_number_of_clusters_night',
                 'locations_mean_len_stay_at_clusters_in_minutes_local_allday',
                 'locations_mean_len_stay_at_clusters_in_minutes_local_morning',
                 'locations_mean_len_stay_at_clusters_in_minutes_local_afternoon',
                 'locations_mean_len_stay_at_clusters_in_minutes_local_evening',
                 'locations_mean_len_stay_at_clusters_in_minutes_local_night',
                 'locations_std_len_stay_at_clusters_in_minutes_local_allday',
                 'locations_std_len_stay_at_clusters_in_minutes_local_morning',
                 'locations_std_len_stay_at_clusters_in_minutes_local_afternoon',
                 'locations_std_len_stay_at_clusters_in_minutes_local_evening',
                 'locations_std_len_stay_at_clusters_in_minutes_local_night',
                 'locations_min_len_stay_at_clusters_in_minutes_local_allday',
                 'locations_min_len_stay_at_clusters_in_minutes_local_morning',
                 'locations_min_len_stay_at_clusters_in_minutes_local_afternoon',
                 'locations_min_len_stay_at_clusters_in_minutes_local_evening',
                 'locations_min_len_stay_at_clusters_in_minutes_local_night',
                 'locations_max_len_stay_at_clusters_in_minutes_local_allday',
                 'locations_max_len_stay_at_clusters_in_minutes_local_morning',
                 'locations_max_len_stay_at_clusters_in_minutes_local_afternoon',
                 'locations_max_len_stay_at_clusters_in_minutes_local_evening',
                 'locations_max_len_stay_at_clusters_in_minutes_local_night',
                 'locations_mean_len_stay_at_clusters_in_minutes_allday',
                 'locations_mean_len_stay_at_clusters_in_minutes_morning',
                 'locations_mean_len_stay_at_clusters_in_minutes_afternoon',
                 'locations_mean_len_stay_at_clusters_in_minutes_evening',
                 'locations_mean_len_stay_at_clusters_in_minutes_night',
                 'locations_std_len_stay_at_clusters_in_minutes_allday',
                 'locations_std_len_stay_at_clusters_in_minutes_morning',
                 'locations_std_len_stay_at_clusters_in_minutes_afternoon',
                 'locations_std_len_stay_at_clusters_in_minutes_evening',
                 'locations_std_len_stay_at_clusters_in_minutes_night',
                 'locations_min_len_stay_at_clusters_in_minutes_allday',
                 'locations_min_len_stay_at_clusters_in_minutes_morning',
                 'locations_min_len_stay_at_clusters_in_minutes_afternoon',
                 'locations_min_len_stay_at_clusters_in_minutes_evening',
                 'locations_min_len_stay_at_clusters_in_minutes_night',
                 'locations_max_len_stay_at_clusters_in_minutes_allday',
                 'locations_max_len_stay_at_clusters_in_minutes_morning',
                 'locations_max_len_stay_at_clusters_in_minutes_afternoon',
                 'locations_max_len_stay_at_clusters_in_minutes_evening',
                 'locations_max_len_stay_at_clusters_in_minutes_night',
                 'locations_time_at_cluster_1_local_allday',
                 'locations_time_at_cluster_1_local_morning',
                 'locations_time_at_cluster_1_local_afternoon',
                 'locations_time_at_cluster_1_local_evening',
                 'locations_time_at_cluster_1_local_night',
                 'locations_time_at_cluster_2_local_allday',
                 'locations_time_at_cluster_2_local_morning',
                 'locations_time_at_cluster_2_local_afternoon',
                 'locations_time_at_cluster_2_local_evening',
                 'locations_time_at_cluster_2_local_night',
                 'locations_time_at_cluster_3_local_allday',
                 'locations_time_at_cluster_3_local_morning',
                 'locations_time_at_cluster_3_local_afternoon',
                 'locations_time_at_cluster_3_local_evening',
                 'locations_time_at_cluster_3_local_night',
                 'locations_time_at_cluster_1_in_group_allday',
                 'locations_time_at_cluster_1_in_group_morning',
                 'locations_time_at_cluster_1_in_group_afternoon',
                 'locations_time_at_cluster_1_in_group_evening',
                 'locations_time_at_cluster_1_in_group_night',
                 'locations_time_at_cluster_2_in_group_allday',
                 'locations_time_at_cluster_2_in_group_morning',
                 'locations_time_at_cluster_2_in_group_afternoon',
                 'locations_time_at_cluster_2_in_group_evening',
                 'locations_time_at_cluster_2_in_group_night',
                 'locations_time_at_cluster_3_in_group_allday',
                 'locations_time_at_cluster_3_in_group_morning',
                 'locations_time_at_cluster_3_in_group_afternoon',
                 'locations_time_at_cluster_3_in_group_evening',
                 'locations_time_at_cluster_3_in_group_night',
                 'locations_time_at_cluster_1_allday',
                 'locations_time_at_cluster_1_morning',
                 'locations_time_at_cluster_1_afternoon',
                 'locations_time_at_cluster_1_evening',
                 'locations_time_at_cluster_1_night',
                 'locations_time_at_cluster_2_allday',
                 'locations_time_at_cluster_2_morning',
                 'locations_time_at_cluster_2_afternoon',
                 'locations_time_at_cluster_2_evening',
                 'locations_time_at_cluster_2_night',
                 'locations_time_at_cluster_3_allday',
                 'locations_time_at_cluster_3_morning',
                 'locations_time_at_cluster_3_afternoon',
                 'locations_time_at_cluster_3_evening',
                 'locations_time_at_cluster_3_night',
                 'locations_speed_mean_meters_per_sec_allday',
                 'locations_speed_mean_meters_per_sec_morning',
                 'locations_speed_mean_meters_per_sec_afternoon',
                 'locations_speed_mean_meters_per_sec_evening',
                 'locations_speed_mean_meters_per_sec_night',
                 'locations_speed_var_meters_per_sec_allday',
                 'locations_speed_var_meters_per_sec_morning',
                 'locations_speed_var_meters_per_sec_afternoon',
                 'locations_speed_var_meters_per_sec_evening',
                 'locations_speed_var_meters_per_sec_night',
                 'locations_total_distance_meters_allday',
                 'locations_total_distance_meters_morning',
                 'locations_total_distance_meters_afternoon',
                 'locations_total_distance_meters_evening',
                 'locations_total_distance_meters_night',
                 'locations_moving_time_percent_local_allday',
                 'locations_moving_time_percent_local_morning',
                 'locations_moving_time_percent_local_afternoon',
                 'locations_moving_time_percent_local_evening',
                 'locations_moving_time_percent_local_night',
                 'locations_moving_time_percent_allday',
                 'locations_moving_time_percent_morning',
                 'locations_moving_time_percent_afternoon',
                 'locations_moving_time_percent_evening',
                 'locations_moving_time_percent_night',
                 'locations_outliers_time_percent_allday',
                 'locations_outliers_time_percent_morning',
                 'locations_outliers_time_percent_afternoon',
                 'locations_outliers_time_percent_evening',
                 'locations_outliers_time_percent_night',
                 'locations_outliers_time_percent_local_allday',
                 'locations_outliers_time_percent_local_morning',
                 'locations_outliers_time_percent_local_afternoon',
                 'locations_outliers_time_percent_local_evening',
                 'locations_outliers_time_percent_local_night',
                 'locations_home_stay_time_percent_10m_allday',
                 'locations_home_stay_time_percent_10m_morning',
                 'locations_home_stay_time_percent_10m_afternoon',
                 'locations_home_stay_time_percent_10m_evening',
                 'locations_home_stay_time_percent_10m_night',
                 'locations_home_stay_time_percent_100m_allday',
                 'locations_home_stay_time_percent_100m_morning',
                 'locations_home_stay_time_percent_100m_afternoon',
                 'locations_home_stay_time_percent_100m_evening',
                 'locations_home_stay_time_percent_100m_night', 
                 'screen_number_samples_screen_allday',
                 'screen_number_samples_screen_morning',
                 'screen_number_samples_screen_afternoon',
                 'screen_number_samples_screen_evening',
                 'screen_number_samples_screen_night',
                 'screen_number_of_minutes_interaction_allday',
                 'screen_number_of_minutes_interaction_morning',
                 'screen_number_of_minutes_interaction_afternoon',
                 'screen_number_of_minutes_interaction_evening',
                 'screen_number_of_minutes_interaction_night',
                 'screen_mean_len_minute_interaction_bout_allday',
                 'screen_mean_len_minute_interaction_bout_morning',
                 'screen_mean_len_minute_interaction_bout_afternoon',
                 'screen_mean_len_minute_interaction_bout_evening',
                 'screen_mean_len_minute_interaction_bout_night',
                 'screen_std_len_minute_interaction_bout_allday',
                 'screen_std_len_minute_interaction_bout_morning',
                 'screen_std_len_minute_interaction_bout_afternoon',
                 'screen_std_len_minute_interaction_bout_evening',
                 'screen_std_len_minute_interaction_bout_night',
                 'screen_min_len_minute_interaction_bout_allday',
                 'screen_min_len_minute_interaction_bout_morning',
                 'screen_min_len_minute_interaction_bout_afternoon',
                 'screen_min_len_minute_interaction_bout_evening',
                 'screen_min_len_minute_interaction_bout_night',
                 'screen_max_len_minute_interaction_bout_allday',
                 'screen_max_len_minute_interaction_bout_morning',
                 'screen_max_len_minute_interaction_bout_afternoon',
                 'screen_max_len_minute_interaction_bout_evening',
                 'screen_max_len_minute_interaction_bout_night',
                 'screen_number_of_minutes_unlock_allday',
                 'screen_number_of_minutes_unlock_morning',
                 'screen_number_of_minutes_unlock_afternoon',
                 'screen_number_of_minutes_unlock_evening',
                 'screen_number_of_minutes_unlock_night',
                 'screen_mean_len_minute_unlock_bout_allday',
                 'screen_mean_len_minute_unlock_bout_morning',
                 'screen_mean_len_minute_unlock_bout_afternoon',
                 'screen_mean_len_minute_unlock_bout_evening',
                 'screen_mean_len_minute_unlock_bout_night',
                 'screen_std_len_minute_unlock_bout_allday',
                 'screen_std_len_minute_unlock_bout_morning',
                 'screen_std_len_minute_unlock_bout_afternoon',
                 'screen_std_len_minute_unlock_bout_evening',
                 'screen_std_len_minute_unlock_bout_night',
                 'screen_min_len_minute_unlock_bout_allday',
                 'screen_min_len_minute_unlock_bout_morning',
                 'screen_min_len_minute_unlock_bout_afternoon',
                 'screen_min_len_minute_unlock_bout_evening',
                 'screen_min_len_minute_unlock_bout_night',
                 'screen_max_len_minute_unlock_bout_allday',
                 'screen_max_len_minute_unlock_bout_morning',
                 'screen_max_len_minute_unlock_bout_afternoon',
                 'screen_max_len_minute_unlock_bout_evening',
                 'screen_max_len_minute_unlock_bout_night',
                 'screen_unlocks_per_minute_allday',
                 'screen_unlocks_per_minute_morning',
                 'screen_unlocks_per_minute_afternoon',
                 'screen_unlocks_per_minute_evening',
                 'screen_unlocks_per_minute_night',
                 'wifi_number_unique_wifi_hotspots_allday',
                 'wifi_number_unique_wifi_hotspots_morning',
                 'wifi_number_unique_wifi_hotspots_afternoon',
                 'wifi_number_unique_wifi_hotspots_evening',
                 'wifi_number_unique_wifi_hotspots_night',
                 'wifi_number_samples_wifi_allday',
                 'wifi_number_samples_wifi_morning',
                 'wifi_number_samples_wifi_afternoon',
                 'wifi_number_samples_wifi_evening',
                 'wifi_number_samples_wifi_night',
                 'totalTimeInBed',
                 'totalMinutesAsleep',
                 'totalSleepRecords',
                 'minutesToFallAsleep_main',
                 'minutesAwake_main',
                 'timeInBed_main',
                 'minutesAsleep_main',
                 'efficiency_main',
                 'duration_main',
                 'minutesAfterWakeup_main',
                 'minutesToFallAsleep_other_aggregated',
                 'minutesAwake_other_aggregated',
                 'timeInBed_other_aggregated',
                 'minutesAsleep_other_aggregated',
                 'efficiency_other_aggregated',
                 'duration_other_aggregated',
                 'minutesAfterWakeup_other_aggregated',
                 'steps']

In [ ]:
summary_sensor = data.groupby(by=['PID', 'day_of_discrimination'])[sensor_metrics].mean()
grand_sensor = summary_sensor.reset_index().groupby(by=['day_of_discrimination'])[sensor_metrics].mean()

In [ ]:
fig, ax = plt.subplots(len(sensor_metrics), 1, figsize=(8, 900))
for ind, metric in enumerate(sensor_metrics):
    grand_sensor[metric].plot(ax=ax[ind])
    ax[ind].set_title(metric)

In [ ]:
pids = set([item[0] for item in summary_sensor.index.values])
fig, ax = plt.subplots(len(pids), 1, figsize=(8, 420))
for ind, pid in enumerate(pids):
    summary_sensor.loc[pid, 'steps'].plot(ax=ax[ind])
    ax[ind].set_xlabel('PID{0:03d}'.format(pid))

In [ ]:
data.groupby(by=['day_of_discrimination']).size()
# as expected number of days labeled as day of discrimiantion is the max followed by next day, next 2 days and so on

In [ ]:
data[(data['same_day'] == 1) & (data['next_day'] == 1)].shape
# shape[0] is the number of consecutive discrimiantion events

In [ ]:
dieoff = pd.DataFrame({'Morning Screen Use' : [0.03, 0.35, 0.01, 0.82, 0.87, 0.52, None], 
'Calls (mean of out/in evening)' : [0.20, 0.14, 0.36, None, 0.71, 0.76, 0.97], 
'Total Minutes Asleep' : [0.16, 0.70, 0.60, 0.01, None, 0.59, 0.20], 
'Time in Bed' : [0.11, 0.77, 0.67, None, None, 0.66, None], 
'Range of Activities during day' : [0.05, 0.20, 0.37, 0.21, None, 0.33, 0.07], 
'Number of steps' : [0.04, 0.83, 0.45, 0.04, 0.74, 0.73, 0.66], 
'Anxious' : [0.03, 0.35, 0.08, 0.99, 0.04, 0.84, 0.32], 
'Depressed' : [0, 0.00, 0.29, 0.6, 0.88, 0.95, 0.40], 
'Frustrated' : [0.0001, 0.03, 0.62, 0.49, 0.17, 0.55, 0.06]})

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
dieoff[0:3].plot(kind='bar', ax=ax)
ax.set_title('Significance of Short-term relations over Days 0-2', fontsize=22)
ax.set_xlabel('Days from Discrimination', fontsize=18)
ax.set_ylabel('Confidence (p-values)', fontsize=18)